Imports:

In [23]:
import torch
from torch import nn  # All neural network modules
from torch import optim  # For optimizers like SGD, Adam, etc.
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment by creating mini batches etc.
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation

We define a new CNN architecture:

In [24]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        # Same Convolution
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        # half the size
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

Set device:

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Define hyperparameters:

In [26]:
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5  # increase to boost performance

Load Data:

In [27]:
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Initialize our NN:

In [28]:
model = CNN().to(device)  # we set the initializer default parameters

Loss function and optimization function:

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Train the NN:

In [30]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # get data to CUDA if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward pass
        pred = model(data)

        # evaluate the loss
        loss = criterion(pred, targets)

        # backward
        # Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
        optimizer.zero_grad()  # set all gradient to zero for each batch
        # Backpropagate the prediction loss with a call to loss.backward().
        # PyTorch deposits the gradients of the loss w.r.t. each parameter.
        loss.backward()  # compute the gradient

        # gradient descent or backward step
        # Performs a single optimization step (parameter update).
        optimizer.step()

Check accuracy on training and testing dataset:

In [31]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("checking accuracy on training data")
    else:
        print("checking accuracy on testing data")

    num_correct = 0
    num_samples = 0
    # TODO
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            res = model(x)
            # the predictions are the indices
            _, pred = res.max(1)
            num_correct += (y == pred).sum()
            num_samples += pred.shape[0]
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/ num_samples * 100:.2f}")

In [32]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

checking accuracy on training data
Got 59137 / 60000 with accuracy 98.56
checking accuracy on testing data
Got 9837 / 10000 with accuracy 98.37
